<a href="https://colab.research.google.com/github/JaJaMalikov/deception/blob/main/beru_gpt2_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎤 Fine-tuning GPT-2 sur les paroles de Bérurier Noir

Ce notebook vous permet d'entraîner un modèle GPT-2 sur un corpus de paroles du groupe Bérurier Noir au format `.jsonl`. Idéalement, utilisez `beru_dataset.jsonl` généré précédemment.

In [1]:
# 📦 Installer les dépendances
!pip install transformers datasets accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# 📁 Importer les bibliothèques
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [3]:
# 📤 Uploader le fichier JSONL
from google.colab import files
uploaded = files.upload()

Saving beru_dataset.jsonl to beru_dataset (1).jsonl


In [4]:
# 📚 Charger le dataset
dataset = load_dataset('json', data_files='beru_dataset.jsonl', split='train')
dataset = dataset.train_test_split(test_size=0.1)

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
# 🔤 Tokenizer et préparation
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example['lyrics'], truncation=True, padding='max_length', max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/66 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [6]:
# 🤖 Charger le modèle GPT2
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [7]:
# ⚙️ Configuration de l'entraînement
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    logging_steps=20,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-fed6499e8cdf>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
# 🏋️‍♂️ Lancer l'entraînement
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: le-larbinator (le-larbinator-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.882500,3.733896
2,3.452100,3.663575
3,3.392700,3.646157


TrainOutput(global_step=99, training_loss=3.4980134289674085, metrics={'train_runtime': 2336.3728, 'train_samples_per_second': 0.085, 'train_steps_per_second': 0.042, 'total_flos': 51735822336000.0, 'train_loss': 3.4980134289674085, 'epoch': 3.0})

In [9]:
# 📝 Génération test
input_prompt = "L'avenir est "
inputs = tokenizer(input_prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_length=200, num_return_sequences=1, do_sample=True, top_k=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


La jeunesse emmerde le niveau
Ain la pieds dans le parlement
En en un pier en plus
Quatre un époutent te défense
Le pieds en brûlée
Le pieds une vie
Dans un tardage seul-que
La pieds des churages in France
Série aux sais par la gens
Les mains de la


In [33]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!cp -r ./results_phase2/checkpoint-333 /content/drive/MyDrive/Bérurier_GPT2_9


OSError: [Errno 107] Transport endpoint is not connected: '/content/drive'

In [11]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "/content/drive/MyDrive/Bérurier_GPT2"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)
model.eval()  # pour éviter d’utiliser les gradients


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [13]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
set_seed(42)  # pour des résultats stables

prompt = "J'ai plus de souvenirs que si j'avais mille ans"
output = generator(prompt, max_length=250, temperature=1.2, num_return_sequences=3, do_sample=True, top_p=0.95, top_k=50)

print(output[0]["generated_text"])


Device set to use cpu


J'ai plus de souvenirs que si j'avais mille ans les tournoi.
Si tes maîtres ruscles du coule en jamais
En toffent la rue des parches qui ont tes détendilles
Je tu les jeunessees sont restient leurs chases du cenet
Leur un se défrent dans l'étude pour la crène au frère
Quatre de l'heur des través en fumérie
Du monde qui jamais fattir
Les toulaux des épats jamais
Jamais sont pour faire, jamais de cêtre plus dans le sang
Je doute les troisques ces chiens et leurs monfils
Sourqu'il ainsi en lieu de déserratoire
Dans la voie est ce n'y avait pas de sonne
Tu ce quelle tout en président sous maître
Y'ai plus de souvenirs que si j'avais mille ans les tournoi


In [31]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import load_dataset
import torch

# 1. Charger le tokenizer et le modèle depuis le checkpoint
checkpoint_path = "/content/drive/MyDrive/Bérurier_GPT2"

tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
model = GPT2LMHeadModel.from_pretrained(checkpoint_path)

# 2. Charger le corpus texte (remplace le chemin si besoin)
dataset = load_dataset('json', data_files='beru_dataset.jsonl', split='train')

# 2. Tokenizer sur la colonne 'lyrics'
def tokenize_function(example):
    result = tokenizer(example["lyrics"], truncation=True, padding="max_length", max_length=256)
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = dataset.map(tokenize_function, batched=False)

# 4. Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results_phase2",       # on garde les nouveaux checkpoints à part
    overwrite_output_dir=True,
    num_train_epochs=9,                  # on continue avec 3 de plus
    per_device_train_batch_size=2,
    save_steps=100,
    save_total_limit=2,
    logging_steps=20,
    logging_dir="./logs",
    report_to="wandb",                   # ou None si tu veux désactiver
)

# 5. Créer le Trainer et relancer l'entraînement (SANS resume_from_checkpoint ici)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

trainer.train()  # simple, efficace, pas d’héritage chelou de l'ancien checkpoint



Map:   0%|          | 0/74 [00:00<?, ? examples/s]

Step,Training Loss
20,3.570300
40,3.448500
60,3.053200
80,3.033300
100,2.960600
120,2.782700
140,2.797300
160,2.700500
180,2.758400
200,2.560600


TrainOutput(global_step=333, training_loss=2.777239790907851, metrics={'train_runtime': 3541.6968, 'train_samples_per_second': 0.188, 'train_steps_per_second': 0.094, 'total_flos': 87010246656000.0, 'train_loss': 2.777239790907851, 'epoch': 9.0})

In [30]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
set_seed(42)  # pour des résultats stables

prompt = "Contre tous les "
output = generator(prompt, max_length=250, temperature=1.3, num_return_sequences=1, do_sample=True, top_p=0.95, top_k=50)

print(output[0]["generated_text"])

Device set to use cpu


Ivre comme étouille de rôle dans sa frie
L'appéchiésé de désclarés à son réchaine
O en vie sa tengue qui noir était une l'attaque des fachaux
Ils avun la fout du sang de seul
Oh ils avec léviens, ils faim à seul de mon ple pleine de lumiande
Nous suivons dans le plus et les femmes dans le poulevard
Chièvre me leur meur meur ils noir des yeux-foutteurants
Qu'il y fais le cette jamais jouées mérons
On me m'aura te te crapeau plus ils étragins les fasaités
Ils un grand dangré fas le camarades
Ils sont dans qu'on feu de cœur-d'meuleur-se
ù chappage en le couleur le tracquerait
De me j'aiment
